### Crytocurrencies analyis

#### Objective
Show statistical 'analysis' on
- Random variable 'winning', where X='BTC' is the event where BTC price increase the most in pct compared to other ccys during in interval
- Random varible 'no trade', where X='BTC' is the event where BTC was not traded during an interval

#### Approach
Just explore data by counting for now 


In [11]:
# Non tech settings
analysis_interval = 15 # seconds
analysis_period_start = '2017-07-01T00:00:00Z'
analysis_period_end = '2017-09-01T00:00:00Z'
ccys = ['BTCUSD', 'ETHUSD', 'EOSUSD', 'XRPUSD', 'LTCUSD']

#### Dependencies

In [12]:
# Tech
import os
import sys
import pandas as pd
import numpy as np
import pendulum

module_path = os.path.join('..')
if module_path not in sys.path:
    sys.path.append(module_path)

from src.models import DeltaPoint, Trade    
from src.db import PgDb

db = PgDb()

###### Retrieve raw price points

In [13]:
# From db
data = [(ccy, db.get_pricepoints(ccy, analysis_period_start, analysis_period_end, interval=analysis_interval)) for ccy in ccys]

DatabaseError: out of memory for query result


In [10]:
# To dataframe
df_dict = {}
for ccy, values in data:
    if not df_dict.get('time'):
        df_dict['time'] = [p['time'] for p in values]
        
    prefix = ccy.replace('USD', '')
    df_dict[f'{prefix}'] = [float(p['pct']) for p in values]
    df_dict[f'{prefix}_trades_count'] = [float(p['n']) for p in values]
    
df = pd.DataFrame(df_dict)
df['winner'] = df[['BTC', 'ETH', 'EOS', 'XRP', 'LTC']].idxmax(axis=1)
df.head(20)

MemoryError: 

###### Winning counts

In [ ]:
winners = df['winner']
counts = winners.value_counts()
probs = counts / winners.size

stats = pd.DataFrame([counts, probs], index=['Count', 'Probability'])
stats.head()

###### No trade counts

In [ ]:
no_trade_list = []
for ccy in ccys:
    prefix = ccy.replace('USD', '')
    column = prefix + '_trades_count'
    trades_count = df[column]
    aggregated_trade_counts = trades_count.value_counts()
    no_trades_count = aggregated_trade_counts.get(0.0)
    if not no_trades_count:
        no_trades_count = 0
        
    ratio = no_trades_count / trades_count.size
        
    no_trade_list.append((prefix, no_trades_count, trades_count.size, ratio))
        
no_trade_df = pd.DataFrame(no_trade_list, columns=['Ccy', 'No Trade Count', 'Total', 'Ratio'])
no_trade_df.head()